In [ ]:
from __future__ import division

from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

class OPT():
    def __init__(self):
        self.epochs=2
        self.image_folder="data/samples"
        self.batch_size=2
        self.model_config_path="config/yolov3_train608_anchor.cfg"
        self.data_config_path="config/coco.data"
        self.weights_path="weights/yolov3.weights"
        self.class_path="data/AI.names"
        self.conf_thres=0.8
        self.nms_thres=0.4
        self.n_cpu==0
        self.img_size=608
        self.checkpoint_interval=1
        self.checkpoint_dir="checkpoints/checkpoint_608"
        
opt = OPT()
print(opt)

cuda = torch.cuda.is_available() and opt.use_cuda

os.makedirs("output", exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)

classes = load_classes(opt.class_path)

# Get data configuration
data_config = parse_data_config(opt.data_config_path)
train_path = data_config["train"]

# Get hyper parameters
hyperparams = parse_model_config(opt.model_config_path)[0]
learning_rate = float(hyperparams["learning_rate"])
momentum = float(hyperparams["momentum"])
decay = float(hyperparams["decay"])
burn_in = int(hyperparams["burn_in"])

# Initiate model
model = Darknet(opt.model_config_path,opt.img_size)
model.load_weights(opt.weights_path)
#model.apply(weights_init_normal)
model.module_list[105]=nn.Sequential(nn.Conv2d(256, 45, kernel_size=(1, 1), stride=(1, 1)))
model.module_list[93]=nn.Sequential(nn.Conv2d(512, 45, kernel_size=(1, 1), stride=(1, 1)))
model.module_list[81]=nn.Sequential(nn.Conv2d(1024, 45, kernel_size=(1, 1), stride=(1, 1)))
model=model_filter(model)
if cuda:
    model = model.cuda()

model.train()

# Get dataloader
dataloader = torch.utils.data.DataLoader(
    TrainDataset(train_path), batch_size=opt.batch_size, shuffle=False)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
for epoch in range(opt.epochs):
    for batch_i, (_, imgs, targets) in enumerate(dataloader):
        imgs = Variable(imgs.type(Tensor))
        targets = Variable(targets.type(Tensor), requires_grad=False)
        optimizer.zero_grad()
        
        loss = model(imgs, targets)
        
        loss.backward()
        optimizer.step()
        if batch_i%2==0:
            print(
                "[Epoch %d/%d, Batch %d/%d] [Losses: x %f, y %f, w %f, h %f, conf %f, cls %f, total %f, recall: %.5f, precision: %.5f]"
                % (
                    epoch,
                    opt.epochs,
                    batch_i,
                    len(dataloader),
                    model.losses["x"],
                    model.losses["y"],
                    model.losses["w"],
                    model.losses["h"],
                    model.losses["conf"],
                    model.losses["cls"],
                    loss.item(),
                    model.losses["recall"],
                    model.losses["precision"],
                )
            )
            
            model.seen += imgs.size(0)
    
    if epoch % opt.checkpoint_interval == 0:
        model.save_weights("%s/%d.weights" % (opt.checkpoint_dir, epoch))
#        if batch_i % 2900 == 0:
#            model.save_weights("%s/%d.weights" % (opt.checkpoint_dir, batch_i))

